In [1]:
from selenium.common.exceptions import NoSuchElementException, ElementClickInterceptedException
from selenium import webdriver
import time
import pandas as pd

In [2]:
def get_jobs(keyword, num_jobs, verbose):
    
    '''Gathers jobs as a dataframe, scraped from Glassdoor'''
    #code source: https://github.com/arapfaik/scraping-glassdoor-selenium/blob/master/glassdoor%20scraping.ipynb(with few changes)

    #Initializing the webdriver
    options = webdriver.ChromeOptions()
    
    #Uncomment the line below if you'd like to scrape without a new Chrome window every time.
    #options.add_argument('headless')
    
    #Change the path to where chromedriver is in your home folder.
    driver = webdriver.Chrome(executable_path="C:/Users/Safoora Naureen/Desktop/DS_Salary_Project/chromedriver", options=options)
    driver.set_window_size(1120, 1000)
    
    #url = 'https://www.glassdoor.co.in/Job/jobs.htm?suggestChosen=false&clickSource=searchBtn&typedKeyword="' + keyword + '"&sc.keyword="' + keyword + '"&locT=&locId=&jobType='
    #url = 'https://www.glassdoor.com/Job/jobs.htm?sc.keyword="' + keyword + '"&locT=C&locId=1147401&locKeyword=San%20Francisco,%20CA&jobType=all&fromAge=-1&minSalary=0&includeNoSalaryJobs=true&radius=100&cityId=-1&minRating=0.0&industryId=-1&sgocId=-1&seniorityType=all&companyId=-1&employerSizes=0&applicationType=0&remoteWorkType=0'
    #url = 'https://www.glassdoor.co.in/Job/us-data-scientist-jobs-SRCH_IL.0,2_IN1_KO3,17.htm'
    #url = 'https://www.glassdoor.co.in/Job/us-data-science-jobs-SRCH_IL.0,2_IN1_KO3,15_IP2.htm'
    url = 'https://www.glassdoor.com/Job/jobs.htm?suggestCount=0&suggestChosen=true&clickSource=searchBtn&typedKeyword=Data&sc.keyword=Data+Scientist&locT=N&locId=1&jobType='
    driver.get(url)
    jobs = []

    while len(jobs) < num_jobs:  #If true, should be still looking for new jobs.

        #Let the page load. Change this number based on your internet speed.
        #Or, wait until the webpage is loaded, instead of hardcoding it.
        time.sleep(4)

        #Test for the "Sign Up" prompt and get rid of it.
        try:
            driver.find_element_by_class_name("selected").click()
        except ElementClickInterceptedException:
            pass

        time.sleep(.1)

        try:
            driver.find_element_by_css_selector("[alt = 'Close']").click()  #clicking to the X.
            print('X click worked')
        except NoSuchElementException:
            print('X click did not work')
            pass

        
        #Going through each job in this page
        job_buttons = driver.find_elements_by_class_name("jl")  #jl for Job Listing. These are the buttons we're going to click.
        for job_button in job_buttons:  

            print("Progress: {}".format("" + str(len(jobs)) + "/" + str(num_jobs)))
            if len(jobs) >= num_jobs:
                break

            #job_button.click()  #You might
            driver.execute_script("arguments[0].click();", job_button)
            time.sleep(4)
            collected_successfully = False
            
            while not collected_successfully:
                try:
                    company_name = driver.find_element_by_xpath('.//div[@class="employerName"]').text
                    location = driver.find_element_by_xpath('.//div[@class="location"]').text
                    job_title = driver.find_element_by_xpath('.//div[contains(@class, "title")]').text
                    job_description = driver.find_element_by_xpath('.//div[@class="jobDescriptionContent desc"]').text
                    collected_successfully = True
                except:
                    time.sleep(5)

            try:
                #salary_estimate = driver.find_element_by_xpath('.//span[@class="gray salary"]').text
                salary_estimate = driver.find_element_by_xpath('.//div[@class="salary"]').text 
            except NoSuchElementException:
                salary_estimate = -1 #You need to set a "not found value. It's important."
            
            try:
                rating = driver.find_element_by_xpath('.//span[@class="rating"]').text
            except NoSuchElementException:
                rating = -1 #You need to set a "not found value. It's important."

            #Printing for debugging
            if verbose:
                print("Job Title: {}".format(job_title))
                print("Salary Estimate: {}".format(salary_estimate))
                print("Job Description: {}".format(job_description[:500]))
                print("Rating: {}".format(rating))
                print("Company Name: {}".format(company_name))
                print("Location: {}".format(location))

            #Going to the Company tab...
            #clicking on this:
            #<div class="tab" data-tab-type="overview"><span>Company</span></div>
            try:
                driver.find_element_by_xpath('.//div[@class="tab" and @data-tab-type="overview"]').click()

                try:
                    #<div class="infoEntity">
                    #    <label>Headquarters</label>
                    #    <span class="value">San Francisco, CA</span>
                    #</div>
                    headquarters = driver.find_element_by_xpath('.//div[@class="infoEntity"]//label[text()="Headquarters"]//following-sibling::*').text
                except NoSuchElementException:
                    headquarters = -1

                try:
                    size = driver.find_element_by_xpath('.//div[@class="infoEntity"]//label[text()="Size"]//following-sibling::*').text
                except NoSuchElementException:
                    size = -1

                try:
                    founded = driver.find_element_by_xpath('.//div[@class="infoEntity"]//label[text()="Founded"]//following-sibling::*').text
                except NoSuchElementException:
                    founded = -1

                try:
                    type_of_ownership = driver.find_element_by_xpath('.//div[@class="infoEntity"]//label[text()="Type"]//following-sibling::*').text
                except NoSuchElementException:
                    type_of_ownership = -1

                try:
                    industry = driver.find_element_by_xpath('.//div[@class="infoEntity"]//label[text()="Industry"]//following-sibling::*').text
                except NoSuchElementException:
                    industry = -1

                try:
                    sector = driver.find_element_by_xpath('.//div[@class="infoEntity"]//label[text()="Sector"]//following-sibling::*').text
                except NoSuchElementException:
                    sector = -1

                try:
                    revenue = driver.find_element_by_xpath('.//div[@class="infoEntity"]//label[text()="Revenue"]//following-sibling::*').text
                except NoSuchElementException:
                    revenue = -1

                try:
                    competitors = driver.find_element_by_xpath('.//div[@class="infoEntity"]//label[text()="Competitors"]//following-sibling::*').text
                except NoSuchElementException:
                    competitors = -1

            except NoSuchElementException:  #Rarely, some job postings do not have the "Company" tab.
                headquarters = -1
                size = -1
                founded = -1
                type_of_ownership = -1
                industry = -1
                sector = -1
                revenue = -1
                competitors = -1

                
            if verbose:
                print("Headquarters: {}".format(headquarters))
                print("Size: {}".format(size))
                print("Founded: {}".format(founded))
                print("Type of Ownership: {}".format(type_of_ownership))
                print("Industry: {}".format(industry))
                print("Sector: {}".format(sector))
                print("Revenue: {}".format(revenue))
                print("Competitors: {}".format(competitors))
                print("@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@")

            jobs.append({"Job Title" : job_title,
            "Salary Estimate" : salary_estimate,
            "Job Description" : job_description,
            "Rating" : rating,
            "Company Name" : company_name,
            "Location" : location,
            "Headquarters" : headquarters,
            "Size" : size,
            "Founded" : founded,
            "Type of ownership" : type_of_ownership,
            "Industry" : industry,
            "Sector" : sector,
            "Revenue" : revenue,
            "Competitors" : competitors})
            #add job to jobs

        #Clicking on the "next page" button
        try:
            driver.find_element_by_xpath('.//li[@class="next"]//a').click()
        except NoSuchElementException:
            print("Scraping terminated before reaching target number of jobs. Needed {}, got {}.".format(num_jobs, len(jobs)))
            break

    return pd.DataFrame(jobs)  #This line converts the dictionary object into a pandas DataFrame.

In [3]:
#This line will open a new chrome window and start the scraping.
df2 = get_jobs("data scientist",400, False)
df2

X click worked
Progress: 0/400
Progress: 1/400
Progress: 2/400
Progress: 3/400
Progress: 4/400
Progress: 5/400
Progress: 6/400
Progress: 7/400
Progress: 8/400
Progress: 9/400
Progress: 10/400
Progress: 11/400
Progress: 12/400
Progress: 13/400
Progress: 14/400
Progress: 15/400
Progress: 16/400
Progress: 17/400
Progress: 18/400
Progress: 19/400
Progress: 20/400
Progress: 21/400
Progress: 22/400
Progress: 23/400
Progress: 24/400
Progress: 25/400
Progress: 26/400
Progress: 27/400
Progress: 28/400
Progress: 29/400
X click did not work
Progress: 30/400
Progress: 31/400
Progress: 32/400
Progress: 33/400
Progress: 34/400
Progress: 35/400
Progress: 36/400
Progress: 37/400
Progress: 38/400
Progress: 39/400
Progress: 40/400
Progress: 41/400
Progress: 42/400
Progress: 43/400
Progress: 44/400
Progress: 45/400
Progress: 46/400
Progress: 47/400
Progress: 48/400
Progress: 49/400
Progress: 50/400
Progress: 51/400
Progress: 52/400
Progress: 53/400
Progress: 54/400
Progress: 55/400
Progress: 56/400
Progr

,Job Title,Salary Estimate,Job Description,Rating,Company Name,Location,Headquarters,Size,Founded,Type of ownership,Industry,Sector,Revenue,Competitors
0,"Data Scientist - Statistics, Early Career",$56K-$97K\n(Glassdoor Est.),*Organization and Job ID**\nJob ID: 310918\n\n...,3.8,PNNL\n3.8,"Richland, WA",-1,1001 to 5000 Employees,1965,Government,Energy,"Oil, Gas, Energy & Utilities",$500 million to $1 billion (USD),-1
1,Data Scientist 1,$57K-$94K\n(Glassdoor Est.),"At NGL, we strive to make data-driven decision...",3.2,National Guardian Life Insurance Company\n3.2,"Madison, WI",-1,201 to 500 Employees,1909,Company - Private,Insurance Carriers,Insurance,$500 million to $1 billion (USD),-1
2,"Software Engineers, Systems Administrators, Sy...",-1,Orbis Operations Hiring Event!Event DetailsDat...,3.7,Orbis Operations\n3.7,"Fort Meade, MD",-1,201 to 500 Employees,-1,Company - Private,Aerospace & Defense,Aerospace & Defense,Unknown / Non-Applicable,-1
3,Population Health Data Analyst,$39K-$69K\n(Glassdoor Est.),Job Summary\n\nThe Healthcare Intelligence Ana...,2.9,Blue Cross Blue Shield of North Dakota\n2.9,"Fargo, ND",-1,501 to 1000 Employees,1940,Company - Private,Insurance Carriers,Insurance,Less than $1 million (USD),-1
4,FCC Data Scientist,$73K-$121K\n(Glassdoor Est.),Requisition Number:\n4199\n\nJob Title:\nFCC D...,2.8,BankUnited\n2.8,"Miami Lakes, FL",-1,1001 to 5000 Employees,2009,Company - Public,Banks & Credit Unions,Finance,$10+ billion (USD),-1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
395,Computer Scientist – Machine Learning,$97K-$169K\n(Glassdoor Est.),Why choose between doing meaningful work and h...,3.2,MITRE\n3.2,"McLean, VA",-1,5001 to 10000 Employees,1958,Nonprofit Organization,Federal Agencies,Government,$1 to $2 billion (USD),-1
396,Transportation Data Analyst III,$55K-$94K\n(Glassdoor Est.),About the Opportunity\nThe Transportation Data...,4.2,Driscoll's\n4.2,"Watsonville, CA",-1,1001 to 5000 Employees,1900,Company - Private,Food Production,Agriculture & Forestry,$2 to $5 billion (USD),-1
397,Data Scientist - Adtech,-1,RESPONSIBILITIES:\n\nKforce is working with a ...,4.1,Kforce\n4.1,"New York, NY",-1,10000+ Employees,1966,Company - Public,Staffing & Outsourcing,Business Services,$1 to $2 billion (USD),-1
398,"Senior Data Scientist, Product Analytics",$128K-$203K\n(Glassdoor Est.),"As Scale’s first Product Data Scientist, you w...",3.3,Scale AI\n3.3,"San Francisco, CA",-1,51 to 200 Employees,2016,Company - Private,Enterprise Software & Network Solutions,Information Technology,Unknown / Non-Applicable,-1


In [4]:
df2.to_csv('glassdoorjobs3.csv')